In [1]:
import csv
import datetime

date_seperator = '/'

auto_start_time = ''

cgm_date_time = []
cgm_values = []
num_days = 0

def compute_average(list):

    avg = [0 for a in range(3)]
    total = [0 for s in range(3)]

    for x in list:
        total[0] = total[0] + x[0]
        total[1] = total[1] + x[1]
        total[2] = total[2] + x[2]

    for r in range(3):
        avg[r] = total[r] / len(list)

    return avg

def linear_interpolate(x0, y0, x1, y1, x):

    return y0 + (x - x0) * (y1 - y0) / (x1 - x0)


with open("InsulinData.csv") as csvfile:
    reader = csv.reader(csvfile)

    reader.__next__()  

    for row in reader:

        if row[16] == 'AUTO MODE ACTIVE PLGM OFF':

            if '/' in row[1]:
                date_seperator = '/'
            else:
                date_seperator = '-'

            date = row[1].split(date_seperator)
            time = row[2].split(':')

            auto_start_time = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(time[0]), int(time[1]),
                                              int(time[2]))

gap_start = -1

with open("CGMData.csv") as csvfile:
    reader = csv.reader(csvfile)
    reader.__next__()  

    i = 0

    cur_date = ''

    for row in reader:

        if '/' in row[1]:
            date_seperator = '/'
        else:
            date_seperator = '-'

        date = row[1].split(date_seperator)
        time = row[2].split(':')

        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(time[0]), int(time[1]), int(time[2]))

        if i == 0:
            cur_date = timestamp.date()
            num_days = num_days + 1

        else:
            if cur_date != timestamp.date():
                cur_date = timestamp.date()
                num_days = num_days + 1

        cgm_date_time.append(timestamp)

        if row[30] != '':
            cgm_values.append(int(row[30]))

            if gap_start != -1:  
                for gi in range(gap_start, i):
                    cgm_values[gi] = linear_interpolate(gap_start - 1, cgm_values[gap_start - 1], i, cgm_values[i], gi)

                gap_start = -1
        else:
            cgm_values.append(-1)
            if gap_start == -1:
                gap_start = i
        i = i + 1


cgm_date_time.reverse()
cgm_values.reverse()

cgm_above_180 = [0 for i in range(3)]
cgm_above_250 = [0 for i in range(3)]
cgm_range = [0 for i in range(3)]
cgm_range_sec = [0 for i in range(3)]
cgm_level_1 = [0 for i in range(3)]
cgm_level_2 = [0 for i in range(3)]

perc_above_180 = []
perc_above_250 = []
perc_range = []
perc_range_sec = []
perc_level_1 = []
perc_level_2 = []

auto_perc_above_180 = []
auto_perc_above_250 = []
auto_perc_range = []
auto_perc_range_sec = []
auto_perc_level_1 = []
auto_perc_level_2 = []

index = -1

cur_day = cgm_date_time[0].date()


for i in range(len(cgm_values)):

    if cur_day != cgm_date_time[i].date() or i == len(cgm_values) - 1:   

        cgm_above_180[2] = cgm_above_180[0] + cgm_above_180[1]
        cgm_above_250[2] = cgm_above_250[0] + cgm_above_250[1]
        cgm_range[2] = cgm_range[0] + cgm_range[1]
        cgm_range_sec[2] = cgm_range_sec[0] + cgm_range_sec[1]
        cgm_level_1[2] = cgm_level_1[0] + cgm_level_1[1]
        cgm_level_2[2] = cgm_level_2[0] + cgm_level_2[1]

        cgm_above_180[:] = [(x / 288.0) * 100 for x in cgm_above_180]
        cgm_above_250[:] = [(x / 288.0) * 100 for x in cgm_above_250]
        cgm_range[:] = [(x / 288.0) * 100 for x in cgm_range]
        cgm_range_sec[:] = [(x / 288.0) * 100 for x in cgm_range_sec]
        cgm_level_1[:] = [(x / 288.0) * 100 for x in cgm_level_1]
        cgm_level_2[:] = [(x / 288.0) * 100 for x in cgm_level_2]

        if cgm_date_time[i] > auto_start_time:
            auto_perc_above_180.append(cgm_above_180)
            auto_perc_above_250.append(cgm_above_250)
            auto_perc_range.append(cgm_range)
            auto_perc_range_sec.append(cgm_range_sec)
            auto_perc_level_1.append(cgm_level_1)
            auto_perc_level_2.append(cgm_level_2)
        else:
            perc_above_180.append(cgm_above_180)
            perc_above_250.append(cgm_above_250)
            perc_range.append(cgm_range)
            perc_range_sec.append(cgm_range_sec)
            perc_level_1.append(cgm_level_1)
            perc_level_2.append(cgm_level_2)

        cgm_above_180 = [0 for i in range(3)]
        cgm_above_250 = [0 for i in range(3)]
        cgm_range = [0 for i in range(3)]
        cgm_range_sec = [0 for i in range(3)]
        cgm_level_1 = [0 for i in range(3)]
        cgm_level_2 = [0 for i in range(3)]

        cur_day = cgm_date_time[i].date()

    if cgm_date_time[i].hour < 6:
        index = 0
    else:
        index = 1

    if cgm_values[i] < 54:
        cgm_level_2[index] = cgm_level_2[index] + 1
    if cgm_values[i] < 70:
        cgm_level_1[index] = cgm_level_1[index] + 1
    if cgm_values[i] >= 70 and cgm_values[i] <= 150:
        cgm_range_sec[index] = cgm_range_sec[index] + 1
    if cgm_values[i] >= 70 and cgm_values[i] <= 180:
        cgm_range[index] = cgm_range[index] + 1
    if cgm_values[i] > 180:
        cgm_above_180[index] = cgm_above_180[index] + 1
    if cgm_values[i] > 250:
        cgm_above_250[index] = cgm_above_250[index] + 1


perc_above_180 = compute_average(perc_above_180)
perc_above_250 = compute_average(perc_above_250)
perc_range = compute_average(perc_range)
perc_range_sec = compute_average(perc_range_sec)
perc_level_1 = compute_average(perc_level_1)
perc_level_2 = compute_average(perc_level_2)

auto_perc_above_180 = compute_average(auto_perc_above_180)
auto_perc_above_250 = compute_average(auto_perc_above_250)
auto_perc_range = compute_average(auto_perc_range)
auto_perc_range_sec = compute_average(auto_perc_range_sec)
auto_perc_level_1 = compute_average(auto_perc_level_1)
auto_perc_level_2 = compute_average(auto_perc_level_2)

with open('Results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(
        [perc_above_180[0], perc_above_250[0], perc_range[0], perc_range_sec[0], perc_level_1[0], perc_level_2[0],
         perc_above_180[1], perc_above_250[1], perc_range[1], perc_range_sec[1], perc_level_1[1], perc_level_2[1],
         perc_above_180[2], perc_above_250[2], perc_range[2], perc_range_sec[2], perc_level_1[2], perc_level_2[2]]
    )

    writer.writerow(
        [auto_perc_above_180[0], auto_perc_above_250[0], auto_perc_range[0], auto_perc_range_sec[0], auto_perc_level_1[0], auto_perc_level_2[0],
         auto_perc_above_180[1], auto_perc_above_250[1], auto_perc_range[1], auto_perc_range_sec[1], auto_perc_level_1[1], auto_perc_level_2[1],
         auto_perc_above_180[2], auto_perc_above_250[2], auto_perc_range[2], auto_perc_range_sec[2], auto_perc_level_1[2], auto_perc_level_2[2]]
    )


